In [1]:
# Improve by ChatGPT & GitHub Copilot
# Running the requierements.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/requierements.ipynb

# Running functions_bar_igw.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/OSSE_borrador/IGW_BAR/functions_bar_igw.ipynb

Extracting only 1 time step

In [ ]:
# Example usage
path_to_mask = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60/1_mesh_mask_TROPICO12_L125_tr21.nc"
path_to_input = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_hf/ssh_hf_"
path_to_save_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/"
path_to_save_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/"
filename = "12.nc"
dx = 10

# Manually load the data
ssh_hf = xr.open_mfdataset(path_to_input + filename)
ds = ssh_hf.rename({'__xarray_dataarray_variable__': 'ssh_hf'}).load()

# Load the mask
mask = xr.open_dataset(path_to_mask, drop_variables={"x", "y"}).tmaskutil[0, :, :].values

# Process the data for all time steps using parallel processing
results = Parallel(n_jobs=-1)(delayed(process_data_for_time_step)(time_step, ds, mask, dx) for time_step in ds.time_counter)

# Separate the results
ssh_bar_list, ssh_igw_list = zip(*results)

# Combine all the results into a single dataset
ssh_bar_combined = xr.concat(ssh_bar_list, dim='time_counter')
ssh_igw_combined = xr.concat(ssh_igw_list, dim='time_counter')

# Extract the prefix from the filename
file_prefix = filename.split('.')[0]

# Save the results
ssh_bar_combined.to_netcdf(f"{path_to_save_bar}ssh_bar_{file_prefix}.nc")
ssh_igw_combined.to_netcdf(f"{path_to_save_igw}ssh_igw_{file_prefix}.nc")

Extracting 12 months at a time

In [3]:
# Define paths
path_to_mask = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60/1_mesh_mask_TROPICO12_L125_tr21.nc"
path_to_input = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_hf/ssh_hf_"
path_to_save_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/"
path_to_save_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/"
dx = 10

# Load the mask
mask = xr.open_dataset(path_to_mask, drop_variables={"x", "y"}).tmaskutil[0, :, :].values

def process_file_for_month(month):
    filename = f"{month:02d}.nc"
    # Load the data
    ssh_hf = xr.open_mfdataset(path_to_input + filename)
    ds = ssh_hf.rename({'__xarray_dataarray_variable__': 'ssh_hf'}).load()

    # Process the data for all time steps using parallel processing
    results = Parallel(n_jobs=-1)(delayed(process_data_for_time_step)(time_step, ds, mask, dx) for time_step in ds.time_counter)

    # Separate the results
    ssh_bar_list, ssh_igw_list = zip(*results)

    # Combine all the results into a single dataset
    ssh_bar_combined = xr.concat(ssh_bar_list, dim='time_counter')
    ssh_igw_combined = xr.concat(ssh_igw_list, dim='time_counter')

    # Extract the prefix from the filename
    file_prefix = filename.split('.')[0]

    # Save the results
    ssh_bar_combined.to_netcdf(f"{path_to_save_bar}ssh_bar_{file_prefix}.nc")
    ssh_igw_combined.to_netcdf(f"{path_to_save_igw}ssh_igw_{file_prefix}.nc")

# Loop through months 01 to 12
for month in range(1, 13):
    process_file_for_month(month)

Testing if extraction is ok

In [7]:
test_igw = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/ssh_igw_01.nc")

In [8]:

test_bar = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/ssh_bar_09.nc")

In [9]:
test_igw

<xarray.Dataset> Size: 250MB
Dimensions:       (y: 213, x: 197, time_counter: 744)
Coordinates:
    lat           (y, x) float32 168kB dask.array<chunksize=(213, 197), meta=np.ndarray>
    lon           (y, x) float32 168kB dask.array<chunksize=(213, 197), meta=np.ndarray>
  * time_counter  (time_counter) datetime64[ns] 6kB 2014-01-01T00:30:00 ... 2...
Dimensions without coordinates: y, x
Data variables:
    ssh_igw       (time_counter, y, x) float64 250MB dask.array<chunksize=(744, 213, 197), meta=np.ndarray>

In [ ]:
# Assuming 'tesst_bar' is already loaded as an xarray.Dataset
time_step = '2014-01-10T07:30:00'

# Select the data for the specified time step
selected_data = test_igw.sel(time_counter=time_step)

# Extract the data variables for plotting
ssh_bar = selected_data.ssh_igw
lat = selected_data.lat
lon = selected_data.lon

# Plot using pcolormesh
plt.figure(figsize=(10, 6))
plt.pcolormesh(lon, lat, ssh_bar, shading='auto')
plt.colorbar(label='Sea Surface Height (m)')
plt.show()

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Assuming 'test_bar' is already loaded as an xarray.Dataset
lat_point = -26
lon_point = 162

# Find the nearest grid point to the specified coordinates
# Flatten the lat and lon arrays for easy indexing
flat_lat = test_igw.lat.values.flatten()
flat_lon = test_igw.lon.values.flatten()

# Calculate the absolute difference between the desired point and all points in the dataset
lat_diff = np.abs(flat_lat - lat_point)
lon_diff = np.abs(flat_lon - lon_point)

# Find the index of the minimum difference
min_index = np.argmin(lat_diff + lon_diff)

# Convert the flat index back to 2D index
y_index, x_index = np.unravel_index(min_index, test_igw.lat.shape)

# Extract the time series data at the nearest point
time_series = test_igw.ssh_igw[:, y_index, x_index]

# Plot the time series
plt.figure(figsize=(12, 6))
time_series.plot()
plt.title(f"Sea Surface Height Time Series at ({lat_point}, {lon_point})")
plt.xlabel("Time")
plt.ylabel("Sea Surface Height (m)")
plt.grid()
plt.show()
